In [2]:
import pandas as pd 
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
att = np.array(pd.read_csv('data/attributes.csv'))
att

array([[0, 'l'],
       [1, 'x'],
       [2, 'x'],
       ...,
       [1497, 'l'],
       [1498, 'f'],
       [1499, 'l']], dtype=object)

In [4]:
G = nx.read_edgelist('data/edges_train.edgelist', data=False, delimiter=',', nodetype=int)
# nx.set_node_attributes(G, att, 'community')

# nx.draw(G, node_size=50, width=0.2)

In [5]:
G.number_of_nodes(), G.number_of_edges()

(1500, 6600)

#### setting:
we have an edgelist containing 6600 links of 1500 nodes and an attribute list containing 1500 nodes with a corresponding level of a categorical attribute variable.
#### aim:
predict the missing links -> should amount to being 7333 links

# Implementation of all possible Link Prediction metrics from networkx
ADD ALL NEW FEATURES TO GETFEATURES

In [13]:
# Input: getFeature(graph, node_i, node_j)
def getFeatures(G, i, j):
    # ressource allocation index
    ra = list(nx.resource_allocation_index(G, [(i, j)]))[0][2]
    
    # jaccard coefficient
    jc = list(nx.jaccard_coefficient(G, [(i, j)]))[0][2]
    
    # adamic adar index
    aa = list(nx.adamic_adar_index(G, [(i, j)]))[0][2]
    
    # preferential attachment
    pa = list(nx.preferential_attachment(G, [(i, j)]))[0][2]
    
    # #common neighbors soundarajan hopcroft
    # sh = list(nx.cn_soundarajan_hopcroft(G, [(i, j)]))[0][2]

    # #ra index soundarajan hopcroft
    # rai = list(nx.ra_index_soundarajan_hopcroft(G, [(i, j)]))[0][2]

    # #within inter cluster
    # wic = list(nx.within_inter_cluster(G, [(i, j)]))[0][2]

    # amount of common neighbors
    cn = len(list(nx.common_neighbors(G, i, j)))

    # check if the nodes are in the same cluster/same attribute
    att_same = 1 if att[i][1] == att[j][1] else 0

    # nx.shortest_path_length(G, i, j)

    #node based features -> could be worth a try
    degree_i = G.degree(i)
    degree_j = G.degree(j)
    clustering_coeff_i = nx.clustering(G, i)
    clustering_coeff_j = nx.clustering(G, j)
    
    # betweenness_centrality_i = nx.betweenness_centrality(G)[i]
    # betweenness_centrality_j = nx.betweenness_centrality(G)[j]

    # Edge betweenness centrality
    # edge_betweenness = nx.edge_betweenness_centrality(G)[(i, j)]


    # katz_centrality_i = nx.katz_centrality_numpy(G)[i]
    # katz_centrality_j = nx.katz_centrality_numpy(G)[j]

    # avg_neighbor_degree_i = nx.average_neighbor_degree(G)[i]
    # avg_neighbor_degree_j = nx.average_neighbor_degree(G)[j]

    # closeness_centrality_i = nx.closeness_centrality(G, i)
    # closeness_centrality_j = nx.closeness_centrality(G, j)

    # pagerank_i = nx.pagerank(G)[i]
    # pagerank_j = nx.pagerank(G)[j]

    # communities = list(nx.community.girvan_newman(G))
    # community_i = [idx_comm for idx_comm, community in enumerate(communities[0]) if i in community][0]
    # community_j = [idx_comm for idx_comm, community in enumerate(communities[0]) if j in community][0]
    # same_community = 1 if community_i == community_j else 0


    # Add the new features to the return list
    return [ra, jc, aa, pa, cn, att_same, 
            # same_community,
            degree_i, degree_j, 
            clustering_coeff_i, clustering_coeff_j, 
            # katz_centrality_i, katz_centrality_j, avg_neighbor_degree_i, avg_neighbor_degree_j, 
            # closeness_centrality_i, closeness_centrality_j, pagerank_i, pagerank_j
    ]


In [7]:
# check if att_same works
att[2][1] == att[0][1]

False

the idea is to create features for all current 

In [8]:
X = []
y = []

for (i, j) in G.edges:
    X.append(getFeatures(G, i, j))
    y.append(1)
    print(i,j)
solInput = pd.read_csv('data/solutionInput.csv')

# set length of 0s to modify ratio (currently set to 1:1)
length_to_modify_ratio = len(X)
length_to_modify_ratio = length_to_modify_ratio * 2

for kk in range(length_to_modify_ratio):
    print("0: " + str(kk))
    #set possible i and j ranges
    possibleiandj = len(att)

    i = np.random.randint(possibleiandj)
    j = np.random.randint(possibleiandj)

    # check if edge already exists -> should be yes
    # check if i and j are the same -> no edge to itself
    # check if i and j are in the solution set/x_test -> should not be set to anything to avoid distorting the solution
    while (i, j) in G.edges or i == j or (i in solInput['int1'] and j in solInput['int2']):
        i = np.random.randint(possibleiandj)
        j = np.random.randint(possibleiandj)

    X.append(getFeatures(G, i, j))
    y.append(0)

KeyboardInterrupt: 

In [8]:
num_rows = len(X)
num_cols = len(X[0]) if X else 0
print(f"Dimensions of features: {num_rows} rows, {num_cols} columns")

Dimensions of features: 19800 rows, 6 columns


In [9]:
X

[[1.5303718734300504, 0.16326530612244897, 6.708297891408715, 3168, 16, 0],
 [0.22348484848484848, 0.046153846153846156, 1.1220111034358629, 960, 3, 0],
 [1.4982719177841128, 0.14423076923076922, 6.434748690039283, 3408, 15, 0],
 [0.352569355527102, 0.09523809523809523, 2.054963393280795, 1008, 6, 0],
 [0.8582042761620227, 0.12658227848101267, 3.9609285722908103, 1968, 10, 0],
 [0.1461038961038961, 0.057971014492753624, 1.1949641953346215, 1200, 4, 0],
 [0.37467532467532466, 0.07692307692307693, 1.8935498449803359, 1056, 5, 0],
 [0.31787238583013233, 0.08620689655172414, 1.7673165915317286, 720, 5, 0],
 [0.4810569105691057, 0.0684931506849315, 2.1048613507607854, 1440, 5, 0],
 [0.015151515151515152, 0.01694915254237288, 0.23868315209103041, 576, 1, 0],
 [0.9327235772357724, 0.09090909090909091, 3.4509097449913106, 1728, 7, 0],
 [0.03847475094469255, 0.03571428571428571, 0.5038767137860016, 480, 2, 0],
 [0.5384848484848485, 0.07936507936507936, 2.1944500928719046, 960, 5, 0],
 [0.099080

In [11]:
solInput = pd.read_csv('data/solutionInput.csv')


In [15]:
X_kaggle = []
X_kaggle = [getFeatures(G, i, j) for i, j in zip(solInput['int1'], solInput['int2'])]

56 396
760 853
340 1137
597 771
1355 1410
240 1066
516 552
502 506
217 1361
38 257
1347 1470
2 24
11 18
269 441
371 449
730 1394
406 1232
151 830
46 1390
1295 1401
771 819
1149 1282
503 537
1293 1433
1009 1024
740 1486
556 1268
856 1425
602 1072
152 516
282 438
729 1495
1434 1490
1251 1271
160 1384
475 580
358 521
370 755
590 947
250 1333
508 728
107 221
16 270
1256 1294
302 413
377 540
1270 1355
764 1447
338 731
108 440
768 879
17 702
303 507
651 652
1324 1409
628 641
1013 1408
1256 1309
1202 1288
1114 1128
834 954
118 881
760 809
502 687
49 788
240 751
579 675
503 549
587 976
198 816
69 1009
9 72
1003 1085
494 1287
35 97
1146 1326
68 730
1047 1131
372 594
1283 1358
447 761
1000 1227
1301 1444
513 520
254 981
408 1013
378 407
1251 1386
290 441
964 1282
172 1193
297 482
771 772
511 526
3 213
568 574
695 1238
328 433
587 679
508 650
324 1089
1105 1380
324 690
293 308
838 1281
564 1292
1005 1010
1010 1079
9 1041
1411 1427
85 794
574 614
11 151
1052 1070
1268 1340
754 939
365 1144
1000 10

In [11]:
pd.DataFrame(X).to_csv('data/x.csv', index=False)
pd.DataFrame(y).to_csv('data/y.csv', index=False)
pd.DataFrame(X_kaggle).to_csv('data/X_kaggle.csv', index=False)